In [2]:
import torch
import pandas as pd
import numpy as np
import os
import torchvision
import torch.nn as nn
import datetime
from torch.utils.data import DataLoader,Dataset
from torchvision.models import resnet50
import torchvision.transforms as transforms
from PIL import Image
import torch.optim as optim
from tensorflow import summary
from torch.utils.data.sampler import SubsetRandomSampler
import matplotlib.pyplot as plt
import cv2
%matplotlib inline

### Loading the dataset with Images code and label under diagnosis column.

In [3]:
df = pd.read_csv('aptos2019-blindness-detection/train.csv')
print(df.shape)
df.head()

(3662, 2)


,id_code,diagnosis
0,000c1434d8d7,2
1,001639a390f0,4
2,0024cdab0c1e,1
3,002c21358ce6,0
4,005b95c28852,0


### Class which creates the dataset in the form of tensor images processed with different transforms acompanied with the labels in the for of label encoded tensors.

In [4]:
class BlindDataset(Dataset):
    def __init__(self,csv,transform):
        self.data = pd.read_csv(csv)
        self.transform = transform
        self.labels = torch.eye(5)[self.data['diagnosis']]
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self,idx):
        image_path = os.path.join('aptos2019-blindness-detection/train_images/'+self.data.loc[idx]['id_code']+'.png')
        image = Image.open(image_path)
        image = self.transform(image)
        label = torch.tensor(self.data.loc[idx]['diagnosis'])
        return {'images':image,'labels':label}

### Defining the preprocessing steps like resizing into (224,224), Random Horizontal Flips, Random Rotations of 45 degrees, Converting to Tensors and Normalizaations

In [5]:
simple_transform = transforms.Compose([transforms.Resize((224,224)),transforms.RandomHorizontalFlip(),transforms.RandomRotation(45),
                                      transforms.ToTensor(),transforms.Normalize([0.496,0.456,0.406],[0.229,0.224,0.225])])

### After class is called it returns a dictionary with image tensors and labels

In [6]:
train_dataset = BlindDataset('../input/aptos2019-blindness-detection/train.csv',simple_transform)

### Generating random indices

In [7]:
data_size = len(train_dataset)
indices = list(range(data_size))
split = int(np.round(0.1*data_size))
train_indices = indices[split:]
valid_indics = indices[:split]

In [8]:
train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(valid_indics)

### Loading the dataset with batch size of 32

In [9]:
train_loader = DataLoader(train_dataset,batch_size=32,sampler=train_sampler)
valid_loader = DataLoader(train_dataset,batch_size=32,sampler=valid_sampler)

### Loading the ResNet50 model without internet and adding 2 linear layers at the end which will be trained according to our dataset.

In [10]:
model = resnet50(pretrained=False)
model.load_state_dict(torch.load('../input/resnet50/resnet50.pth'))
for param in model.parameters():
    param.require_grad = False
    
model.fc = nn.Sequential(
nn.Linear(2048,1024),
    nn.Linear(1024,5))
fc_parameters = model.fc.parameters()
for param in fc_parameters:
    param.require_grad = True
model = model.cuda()

### Defining loss function as Cross entropy loss and optimizer as adam.

In [11]:
criteria = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),lr=0.001)

### Writing the code for training the model

In [12]:
def fit(epochs,model,optimizer,criteria):
    for epoch in range(epochs+1):
        training_loss = 0.0
        validation_loss = 0.0
        correct = 0.0
        total = 0
        print('{}/{} Epochs'.format(epoch+1,epochs))
        
        model.train()
        for batch_idx,d in enumerate(train_loader):
            data = d['images'].cuda()
            target = d['labels'].cuda()
            
            optimizer.zero_grad()
            output = model(data)
            loss = criteria(output,target)
            loss.backward()
            optimizer.step()
            
            training_loss = training_loss + ((1/(batch_idx+1))*(loss.data-training_loss))
            if batch_idx%20==0:
                print('Training loss {}'.format(training_loss))
            pred = output.data.max(1,keepdim=True)[1]
            correct += np.sum(np.squeeze(pred.eq(target.data.view_as(pred))).cpu().numpy())
            total +=data.size(0)
            print('Accuracy on batch {} on Training is {}'.format(batch_idx,(100*correct/total)))

            
        model.eval()
        for batch_idx ,d in enumerate(valid_loader):
            data = d['images'].cuda()
            target = d['labels'].cuda()
            
            output = model(data)
            loss = criteria(output,target)
            
            validation_loss = validation_loss +((1/(batch_idx+1))*(loss.data-validation_loss))
            if batch_idx%20==0:
                print('Validation_loss {}'.format(validation_loss))
            pred = output.data.max(1,keepdim=True)[1]
            correct += np.sum(np.squeeze(pred.eq(target.data.view_as(pred))).cpu().numpy())
            total+=data.size(0)
            print('Validation Accuracy on Batch {} is {}'.format(batch_idx,(100*correct/total)))
            
    return model

In [13]:
fit(10,model,optimizer,criteria)

1/10 Epochs
Training loss 1.5035961866378784
Accuracy on batch 0 on Training is 43.75
Accuracy on batch 1 on Training is 37.5
Accuracy on batch 2 on Training is 44.791666666666664
Accuracy on batch 3 on Training is 42.1875
Accuracy on batch 4 on Training is 48.125
Accuracy on batch 5 on Training is 49.479166666666664
Accuracy on batch 6 on Training is 47.767857142857146
Accuracy on batch 7 on Training is 46.09375
Accuracy on batch 8 on Training is 47.56944444444444
Accuracy on batch 9 on Training is 48.4375
Accuracy on batch 10 on Training is 47.44318181818182
Accuracy on batch 11 on Training is 46.875
Accuracy on batch 12 on Training is 47.83653846153846
Accuracy on batch 13 on Training is 48.660714285714285
Accuracy on batch 14 on Training is 48.541666666666664
Accuracy on batch 15 on Training is 49.4140625
Accuracy on batch 16 on Training is 50.5514705882353
Accuracy on batch 17 on Training is 51.21527777777778
Accuracy on batch 18 on Training is 51.31578947368421
Accuracy on batch 

Accuracy on batch 38 on Training is 71.47435897435898
Accuracy on batch 39 on Training is 71.171875
Training loss 0.8468554019927979
Accuracy on batch 40 on Training is 71.34146341463415
Accuracy on batch 41 on Training is 71.50297619047619
Accuracy on batch 42 on Training is 71.43895348837209
Accuracy on batch 43 on Training is 71.3778409090909
Accuracy on batch 44 on Training is 71.59722222222223
Accuracy on batch 45 on Training is 71.60326086956522
Accuracy on batch 46 on Training is 71.60904255319149
Accuracy on batch 47 on Training is 71.6796875
Accuracy on batch 48 on Training is 71.875
Accuracy on batch 49 on Training is 71.625
Accuracy on batch 50 on Training is 71.50735294117646
Accuracy on batch 51 on Training is 71.33413461538461
Accuracy on batch 52 on Training is 71.34433962264151
Accuracy on batch 53 on Training is 71.18055555555556
Accuracy on batch 54 on Training is 71.07954545454545
Accuracy on batch 55 on Training is 70.81473214285714
Accuracy on batch 56 on Training 

Accuracy on batch 75 on Training is 74.13651315789474
Accuracy on batch 76 on Training is 74.39123376623377
Accuracy on batch 77 on Training is 74.31891025641026
Accuracy on batch 78 on Training is 74.44620253164557
Accuracy on batch 79 on Training is 74.296875
Training loss 0.6721225380897522
Accuracy on batch 80 on Training is 74.30555555555556
Accuracy on batch 81 on Training is 74.35213414634147
Accuracy on batch 82 on Training is 74.28463855421687
Accuracy on batch 83 on Training is 74.21875
Accuracy on batch 84 on Training is 74.26470588235294
Accuracy on batch 85 on Training is 74.23691860465117
Accuracy on batch 86 on Training is 74.31752873563218
Accuracy on batch 87 on Training is 74.25426136363636
Accuracy on batch 88 on Training is 74.40308988764045
Accuracy on batch 89 on Training is 74.30555555555556
Accuracy on batch 90 on Training is 74.38186813186813
Accuracy on batch 91 on Training is 74.32065217391305
Accuracy on batch 92 on Training is 74.32795698924731
Accuracy on 

Validation Accuracy on Batch 9 is 76.82522123893806
Validation Accuracy on Batch 10 is 76.78179824561404
Validation Accuracy on Batch 11 is 76.81594756963408
5/10 Epochs
Training loss 0.778741717338562
Accuracy on batch 0 on Training is 75.0
Accuracy on batch 1 on Training is 75.0
Accuracy on batch 2 on Training is 76.04166666666667
Accuracy on batch 3 on Training is 75.0
Accuracy on batch 4 on Training is 73.75
Accuracy on batch 5 on Training is 75.0
Accuracy on batch 6 on Training is 76.33928571428571
Accuracy on batch 7 on Training is 76.171875
Accuracy on batch 8 on Training is 77.08333333333333
Accuracy on batch 9 on Training is 77.1875
Accuracy on batch 10 on Training is 78.125
Accuracy on batch 11 on Training is 78.125
Accuracy on batch 12 on Training is 78.125
Accuracy on batch 13 on Training is 77.90178571428571
Accuracy on batch 14 on Training is 77.91666666666667
Accuracy on batch 15 on Training is 77.9296875
Accuracy on batch 16 on Training is 77.38970588235294
Accuracy on 

Accuracy on batch 35 on Training is 78.125
Accuracy on batch 36 on Training is 77.87162162162163
Accuracy on batch 37 on Training is 77.71381578947368
Accuracy on batch 38 on Training is 77.88461538461539
Accuracy on batch 39 on Training is 77.890625
Training loss 0.5843360424041748
Accuracy on batch 40 on Training is 77.97256097560975
Accuracy on batch 41 on Training is 77.75297619047619
Accuracy on batch 42 on Training is 77.9796511627907
Accuracy on batch 43 on Training is 77.91193181818181
Accuracy on batch 44 on Training is 77.98611111111111
Accuracy on batch 45 on Training is 78.39673913043478
Accuracy on batch 46 on Training is 78.19148936170212
Accuracy on batch 47 on Training is 78.05989583333333
Accuracy on batch 48 on Training is 77.93367346938776
Accuracy on batch 49 on Training is 77.875
Accuracy on batch 50 on Training is 77.81862745098039
Accuracy on batch 51 on Training is 77.82451923076923
Accuracy on batch 52 on Training is 77.65330188679245
Accuracy on batch 53 on Tr

Accuracy on batch 72 on Training is 78.08219178082192
Accuracy on batch 73 on Training is 78.08277027027027
Accuracy on batch 74 on Training is 78.0
Accuracy on batch 75 on Training is 78.16611842105263
Accuracy on batch 76 on Training is 78.24675324675324
Accuracy on batch 77 on Training is 78.28525641025641
Accuracy on batch 78 on Training is 78.32278481012658
Accuracy on batch 79 on Training is 78.203125
Training loss 0.5607773661613464
Accuracy on batch 80 on Training is 78.04783950617283
Accuracy on batch 81 on Training is 78.04878048780488
Accuracy on batch 82 on Training is 78.04969879518072
Accuracy on batch 83 on Training is 78.08779761904762
Accuracy on batch 84 on Training is 78.125
Accuracy on batch 85 on Training is 78.16133720930233
Accuracy on batch 86 on Training is 78.08908045977012
Accuracy on batch 87 on Training is 78.05397727272727
Accuracy on batch 88 on Training is 78.26544943820225
Accuracy on batch 89 on Training is 78.19444444444444
Accuracy on batch 90 on Tra

Validation Accuracy on Batch 6 is 78.57954545454545
Validation Accuracy on Batch 7 is 78.63175675675676
Validation Accuracy on Batch 8 is 78.59933035714286
Validation Accuracy on Batch 9 is 78.51216814159292
Validation Accuracy on Batch 10 is 78.4265350877193
Validation Accuracy on Batch 11 is 78.399781540142
9/10 Epochs
Training loss 0.33839112520217896
Accuracy on batch 0 on Training is 90.625
Accuracy on batch 1 on Training is 79.6875
Accuracy on batch 2 on Training is 79.16666666666667
Accuracy on batch 3 on Training is 78.90625
Accuracy on batch 4 on Training is 77.5
Accuracy on batch 5 on Training is 76.04166666666667
Accuracy on batch 6 on Training is 78.125
Accuracy on batch 7 on Training is 78.515625
Accuracy on batch 8 on Training is 78.47222222222223
Accuracy on batch 9 on Training is 77.1875
Accuracy on batch 10 on Training is 77.27272727272727
Accuracy on batch 11 on Training is 77.60416666666667
Accuracy on batch 12 on Training is 78.125
Accuracy on batch 13 on Training i

Accuracy on batch 32 on Training is 81.62878787878788
Accuracy on batch 33 on Training is 81.89338235294117
Accuracy on batch 34 on Training is 81.78571428571429
Accuracy on batch 35 on Training is 81.68402777777777
Accuracy on batch 36 on Training is 81.84121621621621
Accuracy on batch 37 on Training is 81.57894736842105
Accuracy on batch 38 on Training is 81.65064102564102
Accuracy on batch 39 on Training is 81.875
Training loss 0.48221465945243835
Accuracy on batch 40 on Training is 81.6310975609756
Accuracy on batch 41 on Training is 81.32440476190476
Accuracy on batch 42 on Training is 81.3953488372093
Accuracy on batch 43 on Training is 81.25
Accuracy on batch 44 on Training is 81.25
Accuracy on batch 45 on Training is 81.0461956521739
Accuracy on batch 46 on Training is 80.91755319148936
Accuracy on batch 47 on Training is 80.79427083333333
Accuracy on batch 48 on Training is 80.80357142857143
Accuracy on batch 49 on Training is 80.875
Accuracy on batch 50 on Training is 80.7598

Accuracy on batch 70 on Training is 74.07570422535211
Accuracy on batch 71 on Training is 74.26215277777777
Accuracy on batch 72 on Training is 74.22945205479452
Accuracy on batch 73 on Training is 74.2820945945946
Accuracy on batch 74 on Training is 74.41666666666667
Accuracy on batch 75 on Training is 74.42434210526316
Accuracy on batch 76 on Training is 74.47240259740259
Accuracy on batch 77 on Training is 74.55929487179488
Accuracy on batch 78 on Training is 74.60443037974683
Accuracy on batch 79 on Training is 74.765625
Training loss 0.6895623207092285
Accuracy on batch 80 on Training is 74.69135802469135
Accuracy on batch 81 on Training is 74.8094512195122
Accuracy on batch 82 on Training is 74.8117469879518
Accuracy on batch 83 on Training is 74.77678571428571
Accuracy on batch 84 on Training is 74.77941176470588
Accuracy on batch 85 on Training is 74.8546511627907
Accuracy on batch 86 on Training is 74.92816091954023
Accuracy on batch 87 on Training is 74.75142045454545
Accurac

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

### Performing same transformations on the prediction dataset.

In [ ]:
class Prediction(Dataset):
    def __init__(self,csv,transform):
        self.data = pd.read_csv(csv)
        self.transform = transform
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self,idx):
        image_path = os.path.join('../input/aptos2019-blindness-detection/test_images/'+self.data.loc[idx]['id_code']+'.png')
        image = Image.open(image_path)
        image = self.transform(image)
        return {'images':image}

In [ ]:
test_dataset = Prediction('../input/aptos2019-blindness-detection/test.csv',simple_transform)

In [ ]:
test_loader = DataLoader(test_dataset)

In [ ]:
prediction = []
for batch_idx,d in enumerate(test_loader):
    data = d['images'].cuda()
    output = model(data)
    output = output.cpu().detach().numpy()
    prediction.append(np.argmax(output))

In [ ]:
sample_submission = pd.read_csv('../input/aptos2019-blindness-detection/sample_submission.csv')

In [ ]:
sample_submission.head()

In [ ]:
sample_submission['diagnosis'] = prediction

In [ ]:
sample_submission.head()

In [ ]:
sample_submission.to_csv('submission.csv',index = False)